Exercise: Build and manage dedicated Docker Image booths
----------------------------------------------------------------------------------------

Now we want to create a Docker image, for this we need a Dockerfile:

In [ ]:
%%bash
cat <<%EOF% >Dockerfile
#
#   Einfache MySQL Umgebung - veraltet nur zu Schulungszwecken!
#
FROM ubuntu:14.04

# root Password setzen, damit kein Dialog erscheint und die Installation haengt!
# Security Problem, besser: https://12factor.net/de/config
RUN echo 'mysql-server mysql-server/root_password password S3cr3tp4ssw0rd' | debconf-set-selections 
RUN echo 'mysql-server mysql-server/root_password_again password S3cr3tp4ssw0rd' | debconf-set-selections 

# Installation
RUN apt-get update && apt-get install -y mysql-server

# mysql config - Port fuer alle Hosts oeffnen 
# Wiederspricht Linux Logik alle Ports zu schliessen. Geht aber nicht anders weil jeder Container seine eigene IP-Adresse hat.
RUN sed -i -e"s/^bind-address\s*=\s*127.0.0.1/bind-address = 0.0.0.0/" /etc/mysql/my.cnf

# Obsolet, siehe Kubernetes Health Probe Pattern
HEALTHCHECK --interval=5m --timeout=3s CMD curl -f localhost:3306 || exit 1

# Obsolet, siehe Kubernetes Ressource: Service
EXPOSE 3306

# Obsolet, siehe Kubernetes PersistenVolumeClaims
VOLUME /var/lib/mysql

CMD ["mysqld"]
%EOF%

To then build the Docker image

In [ ]:
! docker build -t mysql .

We can see the result with `docker image ls`

In [ ]:
! docker image ls

The individual build steps can also be made visible

In [ ]:
! docker history mysql

***
Questions
======

Answer the questions individually or in groups and compare them with the answers


How do I create a container image
<details><summary>Answer</summary>
docker build
</p></details>

---

Which file contains the contents of the container image?
<details><summary>Answer</summary>
Dockerfile
</p></details>

---

Which statement is better (see also [The Twelve-Factor App](https://12factor.net/))?
* a) Dockerfile should preferably include building (CI) and running services, so everything is in one place and the developer can do everything.
* b) The building and execution of services must be strictly separated. So that clean and comprehensible services are created using the CI/CD process.
<details><summary>Answer</summary>
b)
</p></details>

---

Why should versions `tag` of images always be specified?
<details><summary>Answer</summary>
Otherwise `latest` is used and so not sure which version is actually used.
</p></details>

---

What is the difference between `docker save`/`docker load` and `docker export`/`docker import`?
<details><summary>Answer</summary>
save/load ist fÃ¼r Images, export/import fÃ¼r Container.
This makes it possible to exchange container images without Docker Registry, e.g. in a bank.
</p></details>

---